# 📘 Grammar Correction using SQLite + Transformers in Google Colab
Full pipeline: SQLite database + T5 model training + inference

In [ ]:
# ✅ Step 1: Install required libraries
!pip install transformers datasets sentencepiece sacrebleu

In [ ]:
# ✅ Step 2: Create SQLite database and insert data
import sqlite3
import pandas as pd

# Create connection
conn = sqlite3.connect("grammar_data.db")
cursor = conn.cursor()

# Create table
cursor.execute("""
CREATE TABLE IF NOT EXISTS grammar_data (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    incorrect TEXT NOT NULL,
    correct TEXT NOT NULL
)""")

# Insert sample data
data = [
    ("He go to school.", "He goes to school."),
    ("She have a car.", "She has a car."),
    ("I is tired.", "I am tired."),
    ("They was happy.", "They were happy."),
    ("He do homework.", "He does homework.")
]
cursor.executemany("INSERT INTO grammar_data (incorrect, correct) VALUES (?, ?)", data)
conn.commit()

# Read data
df = pd.read_sql("SELECT incorrect, correct FROM grammar_data", conn)
conn.close()
df.head()

,incorrect,correct
0,He go to school.,He goes to school.
1,She have a car.,She has a car.
2,I is tired.,I am tired.
3,They was happy.,They were happy.
4,He do homework.,He does homework.


In [ ]:
# ✅ Step 3: Safe split data into train, val, test
from sklearn.model_selection import train_test_split

if len(df) >= 5:
    train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)

    if len(temp_df) >= 2:
        val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)
    else:
        val_df = temp_df.iloc[:1]
        test_df = temp_df.iloc[1:]
else:
    # fallback split agar data 5 se kam ho
    train_df = df.iloc[:3]
    val_df = df.iloc[3:4]
    test_df = df.iloc[4:]

print("Train set:", len(train_df))
print("Validation set:", len(val_df))
print("Test set:", len(test_df))

Train set: 28
Validation set: 3
Test set: 4


In [ ]:
# ✅ Step 4: Convert to HuggingFace datasets
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

In [ ]:
# ✅ Step 5: Tokenize data
from transformers import AutoTokenizer

model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    inputs = ["fix: " + x for x in examples["incorrect"]]
    model_inputs = tokenizer(inputs, max_length=128, padding="max_length", truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["correct"], max_length=128, padding="max_length", truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/28 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [ ]:
# ✅ Step 6: Load model
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
# ✅ Step 7: Install required libraries
!pip install -U transformers

In [ ]:
# ✅ Step 8: Setup training
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    # Changed 'evaluation_strategy' to 'eval_strategy'
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    predict_with_generate=True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer
)

<ipython-input-171-fafacf8e9fe3>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
# ✅ Step 9: Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,9.669489
2,No log,8.411915
3,No log,7.981674


TrainOutput(global_step=21, training_loss=10.305117652529763, metrics={'train_runtime': 149.9084, 'train_samples_per_second': 0.56, 'train_steps_per_second': 0.14, 'total_flos': 2842177830912.0, 'train_loss': 10.305117652529763, 'epoch': 3.0})

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "prithivida/grammar_error_correcter_v1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def correct_grammar(sentence):
    input_text = "gec: " + sentence
    inputs = tokenizer(input_text, return_tensors="pt", max_length=128, truncation=True)
    outputs = model.generate(
        **inputs,
        max_length=128,
        num_beams=5,
        early_stopping=True
    )
    corrected = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return corrected

# Test example
print(correct_grammar("She go to office."))

She went to the office.


In [ ]:
# ✅ Step 10: Inference / Prediction
def correct_grammar(sentence):
    input_text = "fix: " + sentence
    inputs = tokenizer(input_text, return_tensors="pt")
    outputs = model.generate(**inputs)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

print(correct_grammar("She go to office."))

Fix: She went to the office.


In [ ]:
def correct_grammar(sentence):
    input_text = "gec: " + sentence  # Note: This model expects "gec: " prefix as per its original training
    inputs = tokenizer(input_text, return_tensors="pt", max_length=128, truncation=True)
    outputs = model.generate(**inputs, max_length=128, num_beams=5, early_stopping=True)
    corrected = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return corrected

In [ ]:
test_sentences = [
    "She go to office.",
    "I has a pen.",
    "He are a good boy.",
    "They is playing.",
    "He do his work properly."
]

for s in test_sentences:
    print(f"Incorrect: {s}")
    print(f"Corrected: {correct_grammar(s)}\n")

Incorrect: She go to office.
Corrected: She went to the office.

Incorrect: I has a pen.
Corrected: I have a pen.

Incorrect: He are a good boy.
Corrected: He is a good boy.

Incorrect: They is playing.
Corrected: They are playing.

Incorrect: He do his work properly.
Corrected: He does his work properly.

